In [31]:
from itertools import islice
import random as rnd
import numpy as np

def lcg(n_observations):   
    x_value = 123456789.0    
    a = 101427               
    c = 321                  
    m = (2 ** 16)            
    count = 0
    outFile = open("lcg_out.txt", "w")
   
    while count < n_observations:
        x_value = (a * x_value + c) % m

        #Obtain each number in U[0,1) by diving X_i by m
        writeValue = str(x_value/m)        
        outFile.write(writeValue + "\n")
        count = count + 1
    outFile.close()


def lfsr(n_observations):    
    start_state = 123456789
    # start_state = 1 << 15 | 1
    lfsr = start_state
    period = 0    
    outFile = open("lfsr_out.txt", "w")
    count=0    
    m=(2**16)

    while count< n_observations:
        #taps: 16 15 13 4; feedback polynomial: x^16 + x^15 + x^13 + x^4 + 1
        bit = (lfsr ^ (lfsr >> 1) ^ (lfsr >> 3) ^ (lfsr >> 12)) & 1
        lfsr = (lfsr >> 1) | (bit << 15)
        
        #Obtain each number in U[0,1) by diving X_i by m
        writeValue = str(lfsr/m)
     
        outFile.write(writeValue + "\n")
        period += 1
        count=count+1
        if (lfsr == start_state):
            # print(period)
            break
    outFile.close()
   

In [20]:
# CHI-SQ TEST

def make_subdivisions(input_file):
  subdivisions = {"1": 0,
                  "2": 0,
                  "3": 0,
                  "4": 0,
                  "5": 0,
                  "6": 0,
                  "7": 0,
                  "8": 0,
                  "9": 0,
                  "10": 0}

  with open(input_file, "r") as f:
    data_points = f.readlines()

  for num in data_points:
        num = float(num)
        if num < 0.1:
            subdivisions["1"] += 1
        elif num < 0.2:
            subdivisions["2"] += 1
        elif num < 0.3:
            subdivisions["3"] += 1
        elif num < 0.4:
            subdivisions["4"] += 1
        elif num < 0.5:
            subdivisions["5"] += 1
        elif num < 0.6:
            subdivisions["6"] += 1
        elif num < 0.7:
            subdivisions["7"] += 1
        elif num < 0.8:
            subdivisions["8"] += 1
        elif num < 0.9:
            subdivisions["9"] += 1
        elif num < 1.0:
            subdivisions["10"] += 1

  return subdivisions


def chi_square_uniformity_test( data_set, confidence_level, num_samples ):   
    chi_sq_value = 0.0
    degrees_of_freedom = num_samples - 1
    expected_val = num_samples/10.0

    for observed_val in data_set:
        chi_sq_value += ( pow((expected_val - data_set[observed_val]), 2)/expected_val )

    return chi_sq_value

def chi_sq_significance_test( chi_sq, signif_level):   
    result = "FAIL TO REJECT null hypothesis"
    crit_value = 0.0
    if signif_level == 0.8:
        crit_value = 10118.8246
    elif signif_level == 0.90:
        crit_value = 10181.6616
    elif signif_level == 0.95:
        crit_value = 10233.7489
    else:
        print ("**Invalid Significance Level for Chi Sq***")

    if chi_sq > crit_value:
        result = "REJECT null hypothesis"

    print("\n")
    print ("Significance Level  : " + str(signif_level))
    print ("Chi Sq              : " + str(chi_sq))
    print ("Crit Value          : " + str(crit_value))
    print ("Result is           : " + result)   
    return result

In [29]:
# KS TEST

def get_100_values( data_file ):   
    first_100_vals_as_FLOATS = []
    # grabs first 100 files, as strings with newline endpoints
    with open( data_file, "r" ) as f:
        first_100_vals_as_STRINGS = list(islice(f, 100))

    # transform all values to floats
    for val in first_100_vals_as_STRINGS:
        val = float(val)
        first_100_vals_as_FLOATS.append(val)

    return first_100_vals_as_FLOATS

def get_d_plus_value_for_KS_TEST( data_set, num_samples ):    
    # D+ = max(i/N - R(i))
    d_plus_max = 0
    value_rank_i = 1

    for value in data_set:       
        d_plus_i_value = ( (value_rank_i/num_samples) - value )
        if d_plus_i_value > d_plus_max:
            d_plus_max = d_plus_i_value
        value_rank_i = value_rank_i + 1
    return d_plus_max


def get_d_minus_value_for_KS_TEST( data_set, num_samples ):    
    # D- = max(R(i) - (i -1)/n)
    d_minus_max = 0
    value_rank_i = 1.0
    for value in data_set:       
        substraction_value = ( (value_rank_i - 1.0)/num_samples )
        d_minus_i_value = value - substraction_value

        if d_minus_i_value > d_minus_max:
            d_minus_max = d_minus_i_value
        value_rank_i = value_rank_i + 1

    return d_minus_max

def kolmogorov_smirnov_test( data_set, confidence_level, num_samples ):    
    data_set.sort()
    d_plus = get_d_plus_value_for_KS_TEST(data_set, num_samples)
    print ("D+ VALUE      :"+str(d_plus))

    # D- = max(R(i) - (i -1)/n)
    d_minus = get_d_minus_value_for_KS_TEST(data_set, num_samples)
    print ("D- VALUE      :"+str(d_minus))
    d_value = max(d_plus, d_minus)
    print ("D VALUE(max)  : "+str(d_value))
    return d_value


def ks_significance_test( d_statistic, num_observations, alpha_level ):
    result = "FAIL TO REJECT null hypothesis"
    critical_value = 0

    if alpha_level == 0.1:
        critical_value = 1.22/np.sqrt(num_observations)
    elif alpha_level == 0.05:
        critical_value = 1.36/np.sqrt(num_observations)
    elif alpha_level == 0.01:
        critical_value = 1.63/np.sqrt(num_observations)
    else:
        print ("Invalid alpha level for KS test. Must be: 0.1, 0.05, or 0.01")

    if d_statistic > critical_value:
        result = ("REJECT null hypothesis")

    
    print ("Alpha Level     : " + str(alpha_level))
    print ("D_statistic     : " + str(d_statistic))
    print ("Critical value  : " + str(critical_value))
    print ("Result          : " + result)
    print ("............................")

    return result


In [32]:
n_observations = int(input("Enter Number of Observation : "))

lcg(n_observations)
input_file1 = "lcg_out.txt"

print("_____CHI-SQ TEST FOR LCG_____")
data_points = make_subdivisions(input_file1)
chi_sq_result = chi_square_uniformity_test(data_points, 0, n_observations)
chi_sq_significance_test( chi_sq_result, 0.8 )
chi_sq_significance_test( chi_sq_result, 0.9 )
chi_sq_significance_test( chi_sq_result, 0.95 )

print("\n")
print("_____KS TEST FOR LCG_____")
print("\n")
first_100_values = get_100_values(input_file1)
first_100_values.sort()
ks_result = kolmogorov_smirnov_test(first_100_values,1,100)
ks_significance_test(ks_result,100, 0.1)
ks_significance_test(ks_result,100, 0.05)
ks_significance_test(ks_result,100, 0.01)
print ("Kolmogorov-Smirnov Test Result for D-Value: " + str(ks_result))



Enter Number of Observation : 10
_____CHI-SQ TEST FOR LCG_____


Significance Level  : 0.8
Chi Sq              : 10.0
Crit Value          : 10118.8246
Result is           : FAIL TO REJECT null hypothesis


Significance Level  : 0.9
Chi Sq              : 10.0
Crit Value          : 10181.6616
Result is           : FAIL TO REJECT null hypothesis


Significance Level  : 0.95
Chi Sq              : 10.0
Crit Value          : 10233.7489
Result is           : FAIL TO REJECT null hypothesis


_____KS TEST FOR LCG_____


D+ VALUE      :0
D- VALUE      :0.8446466064453125
D VALUE(max)  : 0.8446466064453125
Alpha Level     : 0.1
D_statistic     : 0.8446466064453125
Critical value  : 0.122
Result          : REJECT null hypothesis
............................
Alpha Level     : 0.05
D_statistic     : 0.8446466064453125
Critical value  : 0.136
Result          : REJECT null hypothesis
............................
Alpha Level     : 0.01
D_statistic     : 0.8446466064453125
Critical value  : 0.1629999999

In [33]:
lfsr(n_observations)
input_file2 = "lfsr_out.txt"
print("\n")
print("_____CHI-SQ TEST FOR LFSR_____")
data_points = make_subdivisions(input_file2)
chi_sq_result = chi_square_uniformity_test(data_points, 0, n_observations)
chi_sq_significance_test( chi_sq_result, 0.8 )
chi_sq_significance_test( chi_sq_result, 0.9 )
chi_sq_significance_test( chi_sq_result, 0.95 )

print("\n")
print("_____KS TEST FOR LFSR_____")
print("\n")
first_100_values = get_100_values(input_file2)
first_100_values.sort()
ks_result = kolmogorov_smirnov_test(first_100_values,1,100)
ks_significance_test(ks_result,100, 0.1)
ks_significance_test(ks_result,100, 0.05)
ks_significance_test(ks_result,100, 0.01)
print ("Kolmogorov-Smirnov Test Result for D-Value: " + str(ks_result))



_____CHI-SQ TEST FOR LFSR_____


Significance Level  : 0.8
Chi Sq              : 10.0
Crit Value          : 10118.8246
Result is           : FAIL TO REJECT null hypothesis


Significance Level  : 0.9
Chi Sq              : 10.0
Crit Value          : 10181.6616
Result is           : FAIL TO REJECT null hypothesis


Significance Level  : 0.95
Chi Sq              : 10.0
Crit Value          : 10233.7489
Result is           : FAIL TO REJECT null hypothesis


_____KS TEST FOR LFSR_____


D+ VALUE      :0
D- VALUE      :941.8105432128906
D VALUE(max)  : 941.8105432128906
Alpha Level     : 0.1
D_statistic     : 941.8105432128906
Critical value  : 0.122
Result          : REJECT null hypothesis
............................
Alpha Level     : 0.05
D_statistic     : 941.8105432128906
Critical value  : 0.136
Result          : REJECT null hypothesis
............................
Alpha Level     : 0.01
D_statistic     : 941.8105432128906
Critical value  : 0.16299999999999998
Result          : REJECT n